In [5]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string



In [6]:
fake = pd.read_csv('Fake.csv')
true=pd.read_csv('True.csv')

In [10]:
fake.head()


,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [11]:
true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [12]:
fake['class']=0
true['class']=1

In [13]:
data=pd.concat([fake,true],axis=0)

In [16]:
data.sample(10)

,text,class
17817,He was given the option to kneel in protest be...,0
14160,BEIRUT (Reuters) - Lebanese Prime Minister Saa...,1
19563,LONDON (Reuters) - Scotland and Wales proposed...,1
3505,"To be a Republican these days, one apparently ...",0
5579,We could already make the assumption that Sena...,0
15055,DUBAI (Reuters) - Iran rejected on Sunday a ca...,1
2083,WASHINGTON (Reuters) - President Donald Trump ...,1
7567,"Tropical Liqueurs, otherwise known as Trops ,...",0
9274,WASHINGTON (Reuters) - Senior U.S. Republicans...,1
17270,,0


In [15]:
data =data.drop(["title","subject","date"],axis=1)

In [17]:
data.reset_index(inplace=True)

In [18]:
data.drop(['index'],axis=1,inplace=True)

In [19]:
data.sample(5)

,text,class
23419,21st Century Wire says Science fiction is now ...,0
26373,CHICAGO (Reuters) - Illinois ended its histori...,1
366,The Republican Party has long claimed that st...,0
4734,The Morning Joe panel just NAILED Donald Trump...,0
18624,You might want to grab a barf bag before you w...,0


In [22]:
def clean_text(text):
    text=text.lower()
    text=re.sub('\[.*?\]',"",text)
    text=re.sub("\\W"," ",text)
    text=re.sub("https?:://\S+|www\.\S+","",text)
    text=re.sub("<.*?>+","",text)
    text=re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text=re.sub("\n","",text)
    text=re.sub("\w*\d\w*","",text)
    return text

In [23]:
data["text"]=data["text"].apply(clean_text)

In [24]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest=train_test_split(x,y,test_size=0.25,random_state=42)

In [29]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [30]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [31]:
prediction=lr.predict(xv_test)
lr.score(xv_test,ytest)

0.9858351893095768

In [32]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [33]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']